# Target Percent Pipeline Algorithm

利用pipeline提供的買賣清單與持股權重進行定期再平衡的演算法。

<span id="menu"></span>
本文件包含以下部份：
1. [Set Environment Variables](#Env)
2. [Investment Universe](#Universe)
3. [Ingest](#Ingest)
4. [Imports](#Imports)
5. [Pipeline](#Pipeline)
6. [Parameters](#Parameters)
7. [Examples](#Examples)
   - [Case 1－調整start_session與end_session](#Case1)
   - [Case 2－調整max_leverage](#Case2)
   - [Case 3－調整tradeday](#Case3)
   - [Case 4－調整stocklist](#Case4)
   - [Case 5－調整allow_short](#Case5)
   - [Case 6－調整cancel_datedelta](#Case6)
   - [Case 7－調整limit_buy_multiplier](#Case7)

<span id="Env"></span>
# 1. Set Environment Variables
[Return to Menu](#menu)

In [1]:
import pandas as pd
import datetime
import tejapi
import os

# set tej_key and base
tej_key = 'your key'
api_base = 'https://api.tej.com.tw'

os.environ['TEJAPI_KEY'] = tej_key
os.environ['TEJAPI_BASE']= api_base

tejapi.ApiConfig.api_key = tej_key  
tejapi.ApiConfig.api_base = api_base


# set benchmark
benchmark=['IR0001']

# set calendar
calendar_name='TEJ_XTAI'

# set bundle name
bundle_name = 'tquant'

# set date
start='2023-06-01'
end='2023-10-03'

# 由文字型態轉為Timestamp，供回測使用
tz = 'UTC'
start_dt, end_dt = pd.Timestamp(start, tz = tz), pd.Timestamp(end, tz = tz)


# 設定os.environ['mdate'] = start+' '+end，供ingest bundle使用
os.environ['mdate'] = start+' '+end

<span id="Universe"></span>
# 2. Investment Universe

台灣50指數成分股

[Return to Menu](#menu)

In [2]:
from zipline.sources.TEJ_Api_Data import get_universe

StockList = get_universe(start, end, idx_id='IX0002')

print(len(StockList))

55


In [3]:
os.environ['ticker'] = ' '.join(StockList + benchmark)

In [4]:
os.environ['ticker']

'1101 1216 1301 1303 1326 1402 1590 1605 2002 2207 2301 2303 2308 2317 2327 2330 2345 2357 2379 2382 2395 2408 2412 2454 2603 2609 2615 2801 2880 2881 2882 2883 2884 2885 2886 2887 2890 2891 2892 2912 3008 3034 3037 3045 3231 3711 4904 4938 5871 5876 5880 6415 6505 6669 9910 IR0001'

<span id="Ingest"></span>
# 3. Ingest
[Return to Menu](#menu)

In [5]:
!zipline ingest -b tquant

Merging daily equity files:


[2023-10-25 07:43:28.516254] INFO: zipline.data.bundles.core: Ingesting tquant.


<span id="Imports"></span>
# 4. Imports & Settings
[Return to Menu](#menu)

In [6]:
import warnings
warnings.filterwarnings('ignore')

In [7]:
from time import time
import numpy as np
import pandas as pd
import empyrical as ep

from logbook import Logger, StderrHandler, INFO, WARNING
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator


from TejToolAPI.TejToolAPI import *

from zipline.api import record

from zipline.utils.calendar_utils import get_calendar
from zipline.utils.run_algo import  (get_transaction_detail,
                                     get_record_vars)

from zipline.pipeline import Pipeline, CustomFactor
from zipline.pipeline.filters import SingleAsset
from zipline.pipeline.factors import RSI

from zipline.TQresearch.tej_pipeline import run_pipeline

# 設定log顯示方式
log_handler = StderrHandler(format_string='[{record.time:%Y-%m-%d %H:%M:%S.%f}]: ' +
                            '{record.level_name}: {record.func_name}: {record.message}',
                            level=INFO)
log_handler.push_application()
log = Logger('Algorithm')

In [8]:
import warnings
warnings.filterwarnings('ignore')

<span id="Pipeline"></span>
# 5. Pipeline

取得`Market_Cap_Dollars`（市值）資料

[Return to Menu](#menu)

In [9]:
from zipline.pipeline.loaders.fundamentals import TQuantFundamentalsPipelineLoader

In [10]:
col = ['Market_Cap_Dollars']

fields = ''
for i in col:
    fields += i
    fields += ' '

os.environ['fields'] = fields

In [11]:
# Ingest Fundamental Bundle
!zipline ingest -b fundamentals

[2023-10-25 07:43:34.247632] INFO: zipline.data.bundles.core: Ingesting fundamentals.


<span id="Parameters"></span>
# 6. Parameters
[Return to Menu](#menu)

**zipline.algo.pipeline_algo.<font color=DeepPink>TargetPercentPipeAlgo</font>** *(self, bundle_name='tquant', start_session=None, end_session=None,*
                                               
*trading_calendar=get_calendar('TEJ_XTAI'), capital_base=1e7, data_frequency='daily', tradeday=None, stocklist=None,*
                                                 
*benchmark='IR0001', volume_limit=0.025, price_impact=0.1, commission_cost=0.001425 + 0.003 / 2, max_leverage=0.8,*
                                                 
*limit_buy_multiplier=None, limit_sell_multiplier=None, allow_short=False, cancel_datedelta=None, custom_weight=False,*
                                                 
*custom_loader=None, pipeline=None, analyze=None, record_vars=None, get_transaction_detail=False)*

利用pipeline提供的買賣清單與持股權重進行定期再平衡的演算法。必要參數僅有`pipeline`。


## Parameters:
---
- **bundle_name** (*str, optional*)－bundle名稱。預設是 **`'tquant'`**。
- **start_session** (*pd.Timestamp or datetime, optional*)－回測起始日期。預設是**bundle中最早的資料日期**。
- **end_session** (*pd.Timestamp or datetime, optional*)－回測結束日期。預設是**bundle中最晚的資料日期**。
- **trading_calendar** (*TradingCalendar, optional*)－
  - 設置交易日曆。預設是 **`get_calendar('TEJ_XTAI')`**。
  - TradingCalendar：`zipline.utils.calendar_utils.TradingCalendar`。
- **capital_base** (*float, optional*)－初始資金額度。預設是**一千萬**。
- **data_frequency** (*{'daily', 'minute'}, optional*)－資料頻率，目前僅支援日頻率 **`'daily'`**。 
- **tradeday** (*list[str], optional*)－交易日期清單，限制只能在這個清單中的日期進行交易。預設是**None**，代表**每日**都交易。
- **stocklist** (*list[str], optional*)－交易清單，限制只能交易這清單中的股票。預設是**None**，代表使用所有bundle中的股票。
- **benchmark** (*str, optional*)－指數名稱，用來與投資組合報酬率比較。預設是`'IR0001'`，代表**台灣發行量加權股價報酬指數**。
- **volume_limit** (*float, optional*)－
  - 限制買賣量佔總交易量的最大百分比，預設 = **2.5%**。
  - 這邊滑價模型採用`zipline.finance.slippage.VolumeShareSlippage`。
- **price_impact** (*float, optional*)－
  - 滑價影響程度，其值越大時，滑價影響程度越大，預設 = **10%**。
  - 這邊滑價模型採用`zipline.finance.slippage.VolumeShareSlippage`。
- **commission_cost** (*float, optional*)－
  - 每交易一元的股票所需支付的固定費用，預設 = **0.001425 + 0.003 / 2**。
  - 這邊模型採用`zipline.finance.commission.PerDollar`。
- **max_leverage** (*float, optional*)－槓桿限制，預設 = 0.8。
- **limit_buy_multiplier** (*float, optional*)－
  - 買進／回補時的limit_price乘數，若有提供則limit_price = 下單時最近一筆收盤價 * `limit_buy_multiplier`。
  - 預設為**None**，代表**不設定**買進／回補時的limit_price。
- **limit_sell_multiplier** (*float, optional*)－
  - 賣出／放空時的limit_price乘數，若有提供則limit_price = 下單時最近一筆收盤價 * `limit_sell_multiplier`。
  - 預設為**None**，代表**不設定**賣出／放空時的limit_price。
- **allow_short** (*bool, optional*)－是否允許放空股票，預設為**False**，代表僅能做多。若設定為**True**，則pipeline中需要有`shorts`欄位。
- **custom_weight** (*bool, optional*)－
  - 是否要使用自訂的加權權數，預設為**False**，代表不使用（等權重加權）。
  - 若設定為**True**，則pipeline中需要有`long_weights`（若`allow_short`=True，則也須有`short_weights`）欄位。
- **cancel_datedelta** (*int, optional*)－訂單幾天內未完全成交就取消。預設是在**下一次再平衡**時取消。
- **custom_loader** (*PipelineLoader , optional*)－
  - 用來取得價量以外資料，預設是**None**，代表不使用價量和`TQDataSet`以外資料。
  - TQDataSet：`zipline.pipeline.data.TQDataSet`
  - 目前支援的`PipelineLoader`：
    - DataFrameLoader（`zipline.pipeline.loaders.frame.DataFrameLoader`）。
- **pipeline** (*Pipeline*)－
  - 要用來產出交易清單或權重的pipeline，為**必要參數**。
  - Pipeline：`zipline.pipeline.Pipeline`
- **analyze** (*callable[(context, pd.DataFrame) -> None], optional*)－
  - 傳入`analyze`函式以用於回測，函式中必須要有`context`與`perf`參數，預設是**None**。
  - 此函式在回測結束時被一次性呼叫，並繪製自訂圖表。
- **record_vars** (*callable[(context, BarData) -> None], optional*)－
  - 傳入`record_vars`函式以用於回測，函式中必須要有`context`與`data`參數，預設是**None**。
  - 此函式在每個交易日結束時被呼叫，並把指定資料紀錄於回測結果的DataFrame中。
- **get_transaction_detail** (*bool, optional*)－
  - 是否產出交易結果，預設為**False**，代表不產出。
  - 若設定為**True**，則可用`algo.positions`、`algo.transactions`、`algo.orders`方式取出交易結果。

## Returns:
---
    algo

## Return type:
---
    zipline.algo.pipeline_algo.TargetPercentPipeAlgo

<span id="Examples"></span>
# 7. Examples
[Return to Menu](#menu)

In [12]:
from zipline.utils.algo_instance import get_algo_instance, set_algo_instance
from zipline.algo.pipeline_algo import *

<span id="Case1"></span>
## Case 1 調整start_session與end_session

[Return to Menu](#menu)

僅調整`start_session`與`end_session`。其餘保持預設值。

以下設定pipeline（`make_pipeline()`），並定義`longs`欄位用來判斷須持有的股票。在`longs`欄位中要持有的股票標記為True，反之標記為False。

In [13]:
from zipline.data import bundles
bundle = bundles.load(bundle_name)

def make_pipeline():
    rsi = RSI()
    longs = rsi.top(2, mask = ~SingleAsset(bundle.asset_finder.lookup_symbol('IR0001', as_of_date=None)))

    return Pipeline(
        
        columns = {
            "longs" : longs,
        }
    )

In [14]:
algo_start = '2023-09-22'
algo_start_dt = pd.Timestamp(algo_start, tz = tz)

result = run_pipeline(make_pipeline(), algo_start, end)
result.query('longs == True')

longs
2023-09-22 00:00:00+00:00 Equity(6 [1590])    True
                          Equity(24 [2603])   True
2023-09-25 00:00:00+00:00 Equity(25 [2609])   True
                          Equity(34 [2886])   True
2023-09-26 00:00:00+00:00 Equity(33 [2885])   True
                          Equity(34 [2886])   True
2023-09-27 00:00:00+00:00 Equity(25 [2609])   True
                          Equity(34 [2886])   True
2023-09-28 00:00:00+00:00 Equity(6 [1590])    True
                          Equity(34 [2886])   True
2023-10-02 00:00:00+00:00 Equity(24 [2603])   True
                          Equity(34 [2886])   True
2023-10-03 00:00:00+00:00 Equity(6 [1590])    True
                          Equity(14 [2327])   True

### 執行演算法
1. 實體化`TargetPercentPipeAlgo`並命名為`algo`。
2. 設定演算法：`set_algo_instance(algo)`
3. 執行演算法，並產出回測報表`stats`：`stats = algo.run()`

In [15]:
# Instantiate TradingAlgorithm
algo = TargetPercentPipeAlgo(
                     start_session=algo_start_dt,
                     end_session=end_dt,
                     pipeline=make_pipeline,
)

# set_algo_instance（zipline\utils\algo_instance.py）
# set context.algorithm = algo
set_algo_instance(algo)

# run
stats = algo.run()

[2023-10-25 07:43:37.950682]: INFO: earn_dividends: Equity(6 [1590]), cash_dividend amount: 13.43905496, pay_date: 2023-10-30, div_owed: 54683.51463224
[2023-10-25 07:43:37.951679]: INFO: handle_split: after split: asset: Equity(6 [1590]), amount: 4067, cost_basis: 984.2, last_sale_price: 981.0
[2023-10-25 07:43:37.952672]: INFO: handle_split: returning cash: 643.28
[2023-10-25 07:43:37.995869]: INFO: handle_simulation_end: Simulated 7 trading days
first open: 2023-09-22 01:01:00+00:00
last close: 2023-10-03 05:30:00+00:00


查看演算法中的參數設定

In [16]:
algo

TargetPercentPipeAlgo(
    sim_params=
SimulationParameters(
    start_session=2023-09-22 00:00:00+00:00,
    end_session=2023-10-03 00:00:00+00:00,
    capital_base=10000000.0,
    data_frequency=daily,
    emission_rate=daily,
    first_open=2023-09-22 01:01:00+00:00,
    last_close=2023-10-03 05:30:00+00:00,
    trading_calendar=<exchange_calendars.exchange_calendar_tejxtai.TEJ_XTAIExchangeCalendar object at 0x00000189FD880D60>
),
    benchmark=IR0001,
    max_leverage=0.8,
    volume_limit=0.025,
    price_impact=0.1,
    commission_cost=0.002925,
    adjust_amount=False,
    limit_buy_multiplier=None,
    limit_sell_multiplier=None,
    allow short or not（if "False" then long only）=False,
    use custom weight or not（if not then "equal weighted"）=False,
    cancel_datedelta（if "None" then cancel at next rebalance date）=None,
    stocklist=['1326', '2308', '5880', '1590', '1303', '2615', '2882', '9910', '6415', '2885', '3231', '2890', '1402', '2301', '5876', '2912', '2883', '2880',

In [17]:
stats.T

,2023-09-22 13:30:00+08:00,2023-09-25 13:30:00+08:00,2023-09-26 13:30:00+08:00,2023-09-27 13:30:00+08:00,2023-09-28 13:30:00+08:00,2023-10-02 13:30:00+08:00,2023-10-03 13:30:00+08:00
period_open,2023-09-22 09:01:00+08:00,2023-09-25 09:01:00+08:00,2023-09-26 09:01:00+08:00,2023-09-27 09:01:00+08:00,2023-09-28 09:01:00+08:00,2023-10-02 09:01:00+08:00,2023-10-03 09:01:00+08:00
period_close,2023-09-22 13:30:00+08:00,2023-09-25 13:30:00+08:00,2023-09-26 13:30:00+08:00,2023-09-27 13:30:00+08:00,2023-09-28 13:30:00+08:00,2023-10-02 13:30:00+08:00,2023-10-03 13:30:00+08:00
short_exposure,0.0,0.0,0.0,0.0,0.0,0.0,0.0
capital_used,0.0,-8067604.150577,-80270.773657,59852.759897,42477.437057,-218249.522863,238352.652208
orders,"[{'id': '75c2846e33224eaab616a81809a35c6a', 'd...","[{'id': '75c2846e33224eaab616a81809a35c6a', 'd...","[{'id': '5383aa2491f44236a27df0d7494d5309', 'd...","[{'id': 'a47494140edc4afdb09ee392b2ec7b99', 'd...","[{'id': '9616738d30324337927613cf903783cf', 'd...","[{'id': '1504b41cb5834e2e88d1586ac95c7d3f', 'd...","[{'id': 'c3299db4d95546df81644b1ff1de53f4', 'd..."
transactions,[],"[{'amount': 34934, 'dt': 2023-09-25 13:30:00+0...","[{'amount': -34934, 'dt': 2023-09-26 13:30:00+...","[{'amount': -86845, 'dt': 2023-09-27 13:30:00+...","[{'amount': -157079, 'dt': 2023-09-28 13:30:00...","[{'amount': -85176, 'dt': 2023-10-02 13:30:00+...","[{'amount': -3977, 'dt': 2023-10-03 13:30:00+0..."
gross_leverage,0.0,0.806304,0.811344,0.804381,0.799241,0.819463,0.793705
positions,[],"[{'sid': Equity(24 [2603]), 'amount': 34934, '...","[{'sid': Equity(25 [2609]), 'amount': 86845, '...","[{'sid': Equity(34 [2886]), 'amount': 103752, ...","[{'sid': Equity(34 [2886]), 'amount': 103703, ...","[{'sid': Equity(34 [2886]), 'amount': 103293, ...","[{'sid': Equity(34 [2886]), 'amount': 102308, ..."
net_leverage,0.0,0.806304,0.811344,0.804381,0.799241,0.819463,0.793705
starting_exposure,0.0,0.0,8044033.0,7965345.5,7861987.25,7780852.3,7880701.8


In [18]:
positions, transactions, orders = get_transaction_detail(stats)

In [19]:
transactions

,sid,symbol,amount,dt,price,order_id,asset,commission
2023-09-25 13:30:00+08:00,24,2603,34934,2023-09-25 13:30:00+08:00,116.000161,75c2846e33224eaab616a81809a35c6a,Equity(24 [2603]),None
2023-09-25 13:30:00+08:00,6,1590,4069,2023-09-25 13:30:00+08:00,981.008992,a44339090ad24fe5a56612bca6d6d699,Equity(6 [1590]),None
2023-09-26 13:30:00+08:00,24,2603,-34934,2023-09-26 13:30:00+08:00,115.499883,5383aa2491f44236a27df0d7494d5309,Equity(24 [2603]),None
2023-09-26 13:30:00+08:00,6,1590,-4067,2023-09-26 13:30:00+08:00,957.984622,85c4509368364c988289f5861c7af793,Equity(6 [1590]),None
2023-09-26 13:30:00+08:00,25,2609,86845,2023-09-26 13:30:00+08:00,45.950385,448fbc9be04648fc824e58f1627f867d,Equity(25 [2609]),None
2023-09-26 13:30:00+08:00,34,2886,105015,2023-09-26 13:30:00+08:00,37.850328,ac9285a46efc437d815649955db287c3,Equity(34 [2886]),None
2023-09-27 13:30:00+08:00,25,2609,-86845,2023-09-27 13:30:00+08:00,45.899836,a47494140edc4afdb09ee392b2ec7b99,Equity(25 [2609]),None
2023-09-27 13:30:00+08:00,33,2885,157079,2023-09-27 13:30:00+08:00,25.150270,cdbfbffe4ac04e5eba622021a4acc276,Equity(33 [2885]),None
2023-09-27 13:30:00+08:00,34,2886,-1263,2023-09-27 13:30:00+08:00,37.700000,8e8363c1464643cf9f10872b3e8cee4e,Equity(34 [2886]),None
2023-09-28 13:30:00+08:00,33,2885,-157079,2023-09-28 13:30:00+08:00,25.049464,9616738d30324337927613cf903783cf,Equity(33 [2885]),None


In [20]:
positions['mv'] = positions['amount'] * positions['last_sale_price']
positions

,sid,symbol,asset,amount,cost_basis,last_sale_price,mv
2023-09-25 13:30:00+08:00,24,2603,Equity(24 [2603]),34934,116.339462,116.00,4052344.00
2023-09-25 13:30:00+08:00,6,1590,Equity(6 [1590]),4069,983.878444,981.00,3991689.00
2023-09-26 13:30:00+08:00,25,2609,Equity(25 [2609]),86845,46.084790,45.95,3990527.75
2023-09-26 13:30:00+08:00,34,2886,Equity(34 [2886]),105015,37.961041,37.85,3974817.75
2023-09-27 13:30:00+08:00,34,2886,Equity(34 [2886]),103752,37.962383,37.70,3911450.40
2023-09-27 13:30:00+08:00,33,2885,Equity(33 [2885]),157079,25.223835,25.15,3950536.85
2023-09-28 13:30:00+08:00,34,2886,Equity(34 [2886]),103703,37.962435,37.70,3909603.10
2023-09-28 13:30:00+08:00,25,2609,Equity(25 [2609]),85176,45.583264,45.45,3871249.20
2023-10-02 13:30:00+08:00,34,2886,Equity(34 [2886]),103293,37.962872,37.60,3883816.80
2023-10-02 13:30:00+08:00,6,1590,Equity(6 [1590]),3977,1007.948011,1005.00,3996885.00


In [21]:
stats.net_leverage

2023-09-22 13:30:00+08:00    0.000000
2023-09-25 13:30:00+08:00    0.806304
2023-09-26 13:30:00+08:00    0.811344
2023-09-27 13:30:00+08:00    0.804381
2023-09-28 13:30:00+08:00    0.799241
2023-10-02 13:30:00+08:00    0.819463
2023-10-03 13:30:00+08:00    0.793705
Name: net_leverage, dtype: float64

<span id="Case2"></span>
## Case 2 調整max_leverage
[Return to Menu](#menu)

接續**Case 1**，多調整`max_leverage=0.70`，其餘與**Case 1**相同。

In [22]:
# Instantiate TradingAlgorithm
algo = TargetPercentPipeAlgo(
                     start_session=algo_start_dt,
                     end_session=end_dt,
                     max_leverage=0.70,
                     pipeline=make_pipeline,
)

# set_algo_instance（zipline\utils\algo_instance.py）
# set context.algorithm = algo
set_algo_instance(algo)

# run
stats = algo.run()

[2023-10-25 07:43:38.238170]: INFO: earn_dividends: Equity(6 [1590]), cash_dividend amount: 13.43905496, pay_date: 2023-10-30, div_owed: 47843.0356576
[2023-10-25 07:43:38.239908]: INFO: handle_split: after split: asset: Equity(6 [1590]), amount: 3558, cost_basis: 984.2, last_sale_price: 981.0
[2023-10-25 07:43:38.239908]: INFO: handle_split: returning cash: 809.04
[2023-10-25 07:43:38.282764]: INFO: handle_simulation_end: Simulated 7 trading days
first open: 2023-09-22 01:01:00+00:00
last close: 2023-10-03 05:30:00+00:00


In [23]:
stats.net_leverage

2023-09-22 13:30:00+08:00    0.000000
2023-09-25 13:30:00+08:00    0.705267
2023-09-26 13:30:00+08:00    0.708485
2023-09-27 13:30:00+08:00    0.703433
2023-09-28 13:30:00+08:00    0.698978
2023-10-02 13:30:00+08:00    0.715954
2023-10-03 13:30:00+08:00    0.694075
Name: net_leverage, dtype: float64

<span id="Case3"></span>
## Case 3 調整tradeday
[Return to Menu](#menu)  

接續**Case 2**，多新增`tradeday`，其餘與**Case 1**相同。

In [24]:
# 設定再平衡日期
freq = 'MS'   # QS-JUL  MS W
_tradeday = list(pd.date_range(start=start_dt, end=end_dt, freq=freq))
tradeday = [get_calendar(calendar_name).next_open(pd.Timestamp(i)).strftime('%Y-%m-%d') if \
           get_calendar(calendar_name).is_session(i)==False else i.strftime('%Y-%m-%d') for i in _tradeday]
tradeday

['2023-06-01', '2023-07-03', '2023-08-01', '2023-09-01', '2023-10-02']

In [25]:
algo_start_dt

Timestamp('2023-09-22 00:00:00+0000', tz='UTC')

In [26]:
# Instantiate TradingAlgorithm
algo = TargetPercentPipeAlgo(
                     start_session=algo_start_dt,
                     end_session=end_dt,
                     capital_base=1e7,                
                     tradeday=tradeday,
                     max_leverage=0.70,
                     pipeline=make_pipeline,
)

# set_algo_instance（zipline\utils\algo_instance.py）
# set context.algorithm = algo
set_algo_instance(algo)

# run
stats = algo.run()

[2023-10-25 07:43:38.429350]: INFO: handle_simulation_end: Simulated 7 trading days
first open: 2023-09-22 01:01:00+00:00
last close: 2023-10-03 05:30:00+00:00


In [27]:
positions, transactions, orders = get_transaction_detail(stats)

In [28]:
transactions

,sid,symbol,amount,dt,price,order_id,asset,commission
2023-10-03 13:30:00+08:00,24,2603,30837,2023-10-03 13:30:00+08:00,110.500039,3b8d6670350a47c49ed6a90b2be79d97,Equity(24 [2603]),None
2023-10-03 13:30:00+08:00,34,2886,93085,2023-10-03 13:30:00+08:00,37.650137,8b2d29d20c52444e80582392e64e754a,Equity(34 [2886]),None


<span id="Case4"></span>
## Case 4 調整stocklist
[Return to Menu](#menu)  


接續**Case 3**，多新增`stocklist`，其餘與**Case 1**相同。  



註1：`stocklist`限制是在pipeline執行完後。  

註2：也可以使用pipeline直接限制股票池。

In [29]:
len(StockList)

55

In [30]:
_StockList = [i for i in StockList if i!='2886']
len(_StockList)

54

In [31]:
# Instantiate TradingAlgorithm
algo = TargetPercentPipeAlgo(
                     start_session=algo_start_dt,
                     end_session=end_dt,
                     capital_base=1e7,                 
                     tradeday=tradeday,
                     stocklist=_StockList,                 
                     max_leverage=0.70,
                     pipeline=make_pipeline,
)

# set_algo_instance（zipline\utils\algo_instance.py）
# set context.algorithm = algo
set_algo_instance(algo)

# run
stats = algo.run()

[2023-10-25 07:43:38.612795]: INFO: handle_simulation_end: Simulated 7 trading days
first open: 2023-09-22 01:01:00+00:00
last close: 2023-10-03 05:30:00+00:00


In [32]:
positions, transactions, orders = get_transaction_detail(stats)

In [33]:
transactions

,sid,symbol,amount,dt,price,order_id,asset,commission
2023-10-03 13:30:00+08:00,24,2603,30837,2023-10-03 13:30:00+08:00,110.500039,452056cfc8244367922356d263dacb9e,Equity(24 [2603]),None


<span id="Case5"></span>
## Case 5 調整allow_short
[Return to Menu](#menu) 

接續**Case 1**，多新增`allow_short=True`，其餘與**Case 1**相同。  


以下設定pipeline（`make_pipeline()`），並定義`shorts`欄位用來判斷須放空的股票。在`shorts`欄位中要放空的股票標記為True，反之標記為False。

In [34]:
def make_pipeline():
    rsi = RSI()
    longs = rsi.top(2, mask = ~SingleAsset(bundle.asset_finder.lookup_symbol('IR0001', as_of_date=None)))
    shorts = rsi.bottom(2, mask = ~SingleAsset(bundle.asset_finder.lookup_symbol('IR0001', as_of_date=None)))

    return Pipeline(
        columns = {
            "longs" : longs,
            "shorts" : shorts
        }
    )

In [35]:
result = run_pipeline(make_pipeline(), algo_start, end)
result.query('(longs == True) | (shorts == True)' )

longs  shorts
2023-09-22 00:00:00+00:00 Equity(6 [1590])    True   False
                          Equity(10 [2301])  False    True
                          Equity(24 [2603])   True   False
                          Equity(39 [2912])  False    True
2023-09-25 00:00:00+00:00 Equity(10 [2301])  False    True
                          Equity(25 [2609])   True   False
                          Equity(31 [2883])  False    True
                          Equity(34 [2886])   True   False
2023-09-26 00:00:00+00:00 Equity(8 [2002])   False    True
                          Equity(10 [2301])  False    True
                          Equity(33 [2885])   True   False
                          Equity(34 [2886])   True   False
2023-09-27 00:00:00+00:00 Equity(10 [2301])  False    True
                          Equity(17 [2357])  False    True
                          Equity(25 [2609])   True   False
                          Equity(34 [2886])   True   False
2023-09-28 00:00:00+00:00 Equity(6 [1590])    True   False
                          Equity(10 [2301])  False    True
                          Equity(34 [2886])   True   False
                          Equity(39 [2912])  False    True
2023-10-02 00:00:00+00:00 Equity(10 [2301])  False    True
                          Equity(24 [2603])   True   False
                          Equity(34 [2886])   True   False
                          Equity(39 [2912])  False    True
2023-10-03 00:00:00+00:00 Equity(2 [1301])   False    True
                          Equity(6 [1590])    True   False
                          Equity(14 [2327])   True   False
                          Equity(30 [2882])  False    True

In [36]:
# Instantiate TradingAlgorithm
algo = TargetPercentPipeAlgo(
                     start_session=algo_start_dt,
                     end_session=end_dt,
                     allow_short=True,
                     pipeline=make_pipeline,
)

# set_algo_instance（zipline\utils\algo_instance.py）
# set context.algorithm = algo
set_algo_instance(algo)

# run
stats = algo.run()

[2023-10-25 07:43:38.816879]: INFO: earn_dividends: Equity(6 [1590]), cash_dividend amount: 13.43905496, pay_date: 2023-10-30, div_owed: 54683.51463224
[2023-10-25 07:43:38.817876]: INFO: handle_split: after split: asset: Equity(6 [1590]), amount: 4067, cost_basis: 984.2, last_sale_price: 981.0
[2023-10-25 07:43:38.818873]: INFO: handle_split: returning cash: 643.28
[2023-10-25 07:43:38.857627]: INFO: handle_data: Cancel_order: current time: 2023-10-02 , created: 2023-09-28 , asset: Equity(39 [2912]), amount: -14713 , filled: -8575
[2023-10-25 07:43:38.882958]: INFO: handle_simulation_end: Simulated 7 trading days
first open: 2023-09-22 01:01:00+00:00
last close: 2023-10-03 05:30:00+00:00


In [37]:
positions, transactions, orders = get_transaction_detail(stats)

In [38]:
# 當天取消
orders.query('(symbol == "2912") & (created.dt.strftime("%Y-%m-%d") == "2023-09-28")')

,sid,symbol,id,dt,reason,created,amount,filled,commission,stop,limit,stop_reached,limit_reached,asset,status
2023-09-28 13:30:00+08:00,39,2912,634588ae10b94a4696a288416d36fcc4,2023-09-28 13:30:00+08:00,None,2023-09-28 13:30:00+08:00,-14713,0,0.000000,None,None,False,False,Equity(39 [2912]),0
2023-10-02 13:30:00+08:00,39,2912,634588ae10b94a4696a288416d36fcc4,2023-10-02 13:30:00+08:00,None,2023-09-28 13:30:00+08:00,-14713,-8575,6596.120842,None,None,False,False,Equity(39 [2912]),2


In [39]:
# vol decimal(13,0) 成交量(千股)  
# 343 * 2.5% = 8.575(千股) = 8575(股) 

tejapi.fastget('TWN/APIPRCD',
               mdate = '2023-10-02',
               coid = ['2912'],
               opts = {'columns':['mdate', 'vol']},
               paginate = True)

,mdate,vol
0,2023-10-02,343.0


In [40]:
positions['mv'] = positions['amount'] * positions['last_sale_price']
positions

,sid,symbol,asset,amount,cost_basis,last_sale_price,mv
2023-09-25 13:30:00+08:00,24,2603,Equity(24 [2603]),34934,116.339462,116.00,4052344.00
2023-09-25 13:30:00+08:00,6,1590,Equity(6 [1590]),4069,983.878444,981.00,3991689.00
2023-09-25 13:30:00+08:00,10,2301,Equity(10 [2301]),-33333,119.648884,120.00,-3999960.00
2023-09-25 13:30:00+08:00,39,2912,Equity(39 [2912]),-15151,263.712312,264.50,-4007439.50
2023-09-26 13:30:00+08:00,10,2301,Equity(10 [2301]),-33176,119.647223,120.00,-3981120.00
2023-09-26 13:30:00+08:00,25,2609,Equity(25 [2609]),86640,46.084788,45.95,3981108.00
2023-09-26 13:30:00+08:00,34,2886,Equity(34 [2886]),104766,37.961039,37.85,3965393.10
2023-09-26 13:30:00+08:00,31,2883,Equity(31 [2883]),-335959,11.765190,11.80,-3964316.20
2023-09-27 13:30:00+08:00,10,2301,Equity(10 [2301]),-32619,119.641229,120.00,-3914280.00
2023-09-27 13:30:00+08:00,34,2886,Equity(34 [2886]),103414,37.962481,37.70,3898707.80


<span id="Case6"></span>
## Case 6 調整cancel_datedelta
[Return to Menu](#menu) 

接續**Case 5**，多新增`cancel_datedelta=2`，其餘與**Case 5**相同。  

In [41]:
# Instantiate TradingAlgorithm
algo = TargetPercentPipeAlgo(
                     start_session=algo_start_dt,
                     end_session=end_dt,
                     allow_short=True,
                     cancel_datedelta=2,
                     pipeline=make_pipeline,
)

# set_algo_instance（zipline\utils\algo_instance.py）
# set context.algorithm = algo
set_algo_instance(algo)

# run
stats = algo.run()

[2023-10-25 07:43:39.090106]: INFO: earn_dividends: Equity(6 [1590]), cash_dividend amount: 13.43905496, pay_date: 2023-10-30, div_owed: 54683.51463224
[2023-10-25 07:43:39.091103]: INFO: handle_split: after split: asset: Equity(6 [1590]), amount: 4067, cost_basis: 984.2, last_sale_price: 981.0
[2023-10-25 07:43:39.092096]: INFO: handle_split: returning cash: 643.28
[2023-10-25 07:43:39.157580]: INFO: handle_simulation_end: Simulated 7 trading days
first open: 2023-09-22 01:01:00+00:00
last close: 2023-10-03 05:30:00+00:00


In [42]:
result = run_pipeline(make_pipeline(), algo_start, end)
result.query('(longs == True) | (shorts == True)' )

longs  shorts
2023-09-22 00:00:00+00:00 Equity(6 [1590])    True   False
                          Equity(10 [2301])  False    True
                          Equity(24 [2603])   True   False
                          Equity(39 [2912])  False    True
2023-09-25 00:00:00+00:00 Equity(10 [2301])  False    True
                          Equity(25 [2609])   True   False
                          Equity(31 [2883])  False    True
                          Equity(34 [2886])   True   False
2023-09-26 00:00:00+00:00 Equity(8 [2002])   False    True
                          Equity(10 [2301])  False    True
                          Equity(33 [2885])   True   False
                          Equity(34 [2886])   True   False
2023-09-27 00:00:00+00:00 Equity(10 [2301])  False    True
                          Equity(17 [2357])  False    True
                          Equity(25 [2609])   True   False
                          Equity(34 [2886])   True   False
2023-09-28 00:00:00+00:00 Equity(6 [1590])    True   False
                          Equity(10 [2301])  False    True
                          Equity(34 [2886])   True   False
                          Equity(39 [2912])  False    True
2023-10-02 00:00:00+00:00 Equity(10 [2301])  False    True
                          Equity(24 [2603])   True   False
                          Equity(34 [2886])   True   False
                          Equity(39 [2912])  False    True
2023-10-03 00:00:00+00:00 Equity(2 [1301])   False    True
                          Equity(6 [1590])    True   False
                          Equity(14 [2327])   True   False
                          Equity(30 [2882])  False    True

In [43]:
positions, transactions, orders = get_transaction_detail(stats)

In [44]:
orders.query('(symbol == "2912") & (created.dt.strftime("%Y-%m-%d") == "2023-09-28")')

,sid,symbol,id,dt,reason,created,amount,filled,commission,stop,limit,stop_reached,limit_reached,asset,status
2023-09-28 13:30:00+08:00,39,2912,1df8aa761f0e4ef0862d6426c62fe1e0,2023-09-28 13:30:00+08:00,None,2023-09-28 13:30:00+08:00,-14713,0,0.000000,None,None,False,False,Equity(39 [2912]),0
2023-10-02 13:30:00+08:00,39,2912,1df8aa761f0e4ef0862d6426c62fe1e0,2023-10-02 13:30:00+08:00,None,2023-09-28 13:30:00+08:00,-14713,-8575,6596.120842,None,None,False,False,Equity(39 [2912]),0
2023-10-03 13:30:00+08:00,39,2912,1df8aa761f0e4ef0862d6426c62fe1e0,2023-10-03 13:30:00+08:00,None,2023-09-28 13:30:00+08:00,-14713,-14713,11299.949997,None,None,False,False,Equity(39 [2912]),1


In [45]:
# vol decimal(13,0) 成交量(千股)  
# 10/02：343 * 2.5% =  8.575(千股) =  8575(股) 
# 10/03：808 * 2.5% = 20.200(千股) = 20200(股) 

tejapi.fastget('TWN/APIPRCD',
               mdate = {'gte':'2023-10-02','lte':'2023-10-03'},
               coid = ['2912'],
               opts = {'columns':['mdate', 'vol']},
               paginate = True)

,mdate,vol
0,2023-10-02,343.0
1,2023-10-03,808.0


In [46]:
positions['mv'] = positions['amount'] * positions['last_sale_price']
positions

,sid,symbol,asset,amount,cost_basis,last_sale_price,mv
2023-09-25 13:30:00+08:00,24,2603,Equity(24 [2603]),34934,116.339462,116.00,4052344.00
2023-09-25 13:30:00+08:00,6,1590,Equity(6 [1590]),4069,983.878444,981.00,3991689.00
2023-09-25 13:30:00+08:00,10,2301,Equity(10 [2301]),-33333,119.648884,120.00,-3999960.00
2023-09-25 13:30:00+08:00,39,2912,Equity(39 [2912]),-15151,263.712312,264.50,-4007439.50
2023-09-26 13:30:00+08:00,10,2301,Equity(10 [2301]),-33176,119.647223,120.00,-3981120.00
2023-09-26 13:30:00+08:00,25,2609,Equity(25 [2609]),86640,46.084788,45.95,3981108.00
2023-09-26 13:30:00+08:00,34,2886,Equity(34 [2886]),104766,37.961039,37.85,3965393.10
2023-09-26 13:30:00+08:00,31,2883,Equity(31 [2883]),-335959,11.765190,11.80,-3964316.20
2023-09-27 13:30:00+08:00,10,2301,Equity(10 [2301]),-32619,119.641229,120.00,-3914280.00
2023-09-27 13:30:00+08:00,34,2886,Equity(34 [2886]),103414,37.962481,37.70,3898707.80


<span id="Case7"></span>
## Case 7 調整limit_buy_multiplier
[Return to Menu](#menu) 

接續**Case 6**，多設定`limit_buy_multiplier=1.015`，其餘與**Case 6**相同。

In [47]:
result = run_pipeline(make_pipeline(), algo_start, end)
result.query('(longs == True) | (shorts == True)' )

longs  shorts
2023-09-22 00:00:00+00:00 Equity(6 [1590])    True   False
                          Equity(10 [2301])  False    True
                          Equity(24 [2603])   True   False
                          Equity(39 [2912])  False    True
2023-09-25 00:00:00+00:00 Equity(10 [2301])  False    True
                          Equity(25 [2609])   True   False
                          Equity(31 [2883])  False    True
                          Equity(34 [2886])   True   False
2023-09-26 00:00:00+00:00 Equity(8 [2002])   False    True
                          Equity(10 [2301])  False    True
                          Equity(33 [2885])   True   False
                          Equity(34 [2886])   True   False
2023-09-27 00:00:00+00:00 Equity(10 [2301])  False    True
                          Equity(17 [2357])  False    True
                          Equity(25 [2609])   True   False
                          Equity(34 [2886])   True   False
2023-09-28 00:00:00+00:00 Equity(6 [1590])    True   False
                          Equity(10 [2301])  False    True
                          Equity(34 [2886])   True   False
                          Equity(39 [2912])  False    True
2023-10-02 00:00:00+00:00 Equity(10 [2301])  False    True
                          Equity(24 [2603])   True   False
                          Equity(34 [2886])   True   False
                          Equity(39 [2912])  False    True
2023-10-03 00:00:00+00:00 Equity(2 [1301])   False    True
                          Equity(6 [1590])    True   False
                          Equity(14 [2327])   True   False
                          Equity(30 [2882])  False    True

In [48]:
# Instantiate TradingAlgorithm
algo = TargetPercentPipeAlgo(
                     start_session=algo_start_dt,
                     end_session=end_dt,
                     limit_buy_multiplier=1.015,
                     allow_short=True,
                     cancel_datedelta=2,
                     pipeline=make_pipeline,
)

# set_algo_instance（zipline\utils\algo_instance.py）
# set context.algorithm = algo
set_algo_instance(algo)

# run
stats = algo.run()

[2023-10-25 07:43:39.431486]: INFO: earn_dividends: Equity(6 [1590]), cash_dividend amount: 13.43905496, pay_date: 2023-10-30, div_owed: 54683.51463224
[2023-10-25 07:43:39.432487]: INFO: handle_split: after split: asset: Equity(6 [1590]), amount: 4067, cost_basis: 984.2, last_sale_price: 981.0
[2023-10-25 07:43:39.432487]: INFO: handle_split: returning cash: 643.28
[2023-10-25 07:43:39.491599]: INFO: handle_data: Cancel_order: current time: 2023-10-03,
                              due to : created>=current time + cancel_datedelta(2 days),
                              created: 2023-09-28, asset: Equity(6 [1590]), amount: 3945, filled: 0'
[2023-10-25 07:43:39.498576]: INFO: handle_simulation_end: Simulated 7 trading days
first open: 2023-09-22 01:01:00+00:00
last close: 2023-10-03 05:30:00+00:00


In [49]:
positions, transactions, orders = get_transaction_detail(stats)

In [50]:
orders

,sid,symbol,id,dt,reason,created,amount,filled,commission,stop,limit,stop_reached,limit_reached,asset,status
2023-09-22 13:30:00+08:00,24,2603,8672fef3043648f2897d5d739420bd1b,2023-09-22 13:30:00+08:00,None,2023-09-22 13:30:00+08:00,34934,0,0.000000,None,116.21,False,False,Equity(24 [2603]),0
2023-09-22 13:30:00+08:00,6,1590,485253f5d4004aa989a395739578ead1,2023-09-22 13:30:00+08:00,None,2023-09-22 13:30:00+08:00,4069,0,0.000000,None,997.74,False,False,Equity(6 [1590]),0
2023-09-22 13:30:00+08:00,10,2301,8b9da1a12dbf4ba6ba4bb8250fb38d09,2023-09-22 13:30:00+08:00,None,2023-09-22 13:30:00+08:00,-33333,0,0.000000,None,NaN,False,False,Equity(10 [2301]),0
2023-09-22 13:30:00+08:00,39,2912,92c1930d68034136b9c91b2a12aa1a83,2023-09-22 13:30:00+08:00,None,2023-09-22 13:30:00+08:00,-15151,0,0.000000,None,NaN,False,False,Equity(39 [2912]),0
2023-09-25 13:30:00+08:00,24,2603,8672fef3043648f2897d5d739420bd1b,2023-09-25 13:30:00+08:00,None,2023-09-22 13:30:00+08:00,34934,34934,11853.122701,None,116.21,False,True,Equity(24 [2603]),1
2023-09-25 13:30:00+08:00,6,1590,485253f5d4004aa989a395739578ead1,2023-09-25 13:30:00+08:00,None,2023-09-22 13:30:00+08:00,4069,4069,11675.797349,None,997.74,False,True,Equity(6 [1590]),1
2023-09-25 13:30:00+08:00,10,2301,8b9da1a12dbf4ba6ba4bb8250fb38d09,2023-09-25 13:30:00+08:00,None,2023-09-22 13:30:00+08:00,-33333,-33333,11699.871627,None,NaN,False,False,Equity(10 [2301]),1
2023-09-25 13:30:00+08:00,39,2912,92c1930d68034136b9c91b2a12aa1a83,2023-09-25 13:30:00+08:00,None,2023-09-22 13:30:00+08:00,-15151,-15151,11721.137169,None,NaN,False,False,Equity(39 [2912]),1
2023-09-25 13:30:00+08:00,24,2603,295a141ed4d04d5aaf879f1e021ea04f,2023-09-25 13:30:00+08:00,None,2023-09-25 13:30:00+08:00,-34934,0,0.000000,None,NaN,False,False,Equity(24 [2603]),0
2023-09-25 13:30:00+08:00,6,1590,c3498defd9a44e9397b325ad2fe551db,2023-09-25 13:30:00+08:00,None,2023-09-25 13:30:00+08:00,-4069,0,0.000000,None,NaN,False,False,Equity(6 [1590]),0


In [51]:
orders.query('(symbol == "1590") & (created.dt.strftime("%Y-%m-%d") == "2023-09-28")')

,sid,symbol,id,dt,reason,created,amount,filled,commission,stop,limit,stop_reached,limit_reached,asset,status
2023-09-28 13:30:00+08:00,6,1590,74b1a456f47645d291a14c36a827e202,2023-09-28 13:30:00+08:00,None,2023-09-28 13:30:00+08:00,3945,0,0.0,None,993.68,False,False,Equity(6 [1590]),0
2023-10-03 13:30:00+08:00,6,1590,74b1a456f47645d291a14c36a827e202,2023-10-03 13:30:00+08:00,None,2023-09-28 13:30:00+08:00,3945,0,0.0,None,993.68,False,False,Equity(6 [1590]),2


In [52]:
# 9/28 979 * 1.015 = 993.685

tejapi.fastget('TWN/APIPRCD',
               mdate = {'gte':'2023-09-28','lte':'2023-10-03'},
               coid = ['1590'],
               opts = {'columns':['coid','mdate', 'close_d']},
               paginate = True)

,coid,mdate,close_d
0,1590,2023-09-28,979.0
1,1590,2023-10-02,1005.0
2,1590,2023-10-03,998.0


[Return to Menu](#menu) 